In [1]:
setwd('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1009. NC ENVRIOSCAN/P1009.3. Analyses/P1009.3.2. Data Cleaning/Input')
Output = ('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1009. NC ENVRIOSCAN/P1009.3. Analyses/P1009.3.2. Data Cleaning/Output')
cur_date = '010523'

library(readxl)
library(tidyverse)

# reading in files
county_df = data.frame(read_excel("nc.xls"))
whp_df = data.frame(read_excel("Wildfire_Hazard_010323.xlsx", sheet = 2))
# making the last 2 cols numeric
whp_df[,3:4] = apply(whp_df[,3:4], 2, as.numeric)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.3.6      ✔ purrr   0.3.4 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.4.1 
✔ readr   2.1.2      ✔ forcats 0.5.2 
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Warning message in apply(whp_df[, 3:4], 2, as.numeric):
“NAs introduced by coercion”
Warning message in apply(whp_df[, 3:4], 2, as.numeric):
“NAs introduced by coercion”


In [2]:
head(county_df)
head(whp_df)

,STATE,LOGRECNO,GEO_ID,GEOGRAPHY.NAME
,<chr>,<chr>,<chr>,<chr>
1,NC,0000001,04000US37,North Carolina
2,NC,0000002,04001US37,North Carolina -- Urban
3,NC,0000003,04043US37,North Carolina -- Rural
4,NC,0000004,040A0US37,North Carolina -- In metropolitan or micropolitan statistical area
5,NC,0000005,040C0US37,North Carolina -- In metropolitan statistical area
6,NC,0000006,040C1US37,North Carolina -- In metropolitan statistical area -- in principal city


,GEO_ID,Census_Tract,Avg_ACRES,Wildfire_Hazard_Potential_Mean
,<chr>,<dbl>,<dbl>,<dbl>
1,14000US37119005916,59.16,349323.3,139.2436
2,14000US37119003808,38.08,349323.3,139.2436
3,14000US37119005918,59.18,349323.3,139.2436
4,14000US37119005914,59.14,349323.3,139.2436
5,14000US37071031004,310.04,233025.3,188.1261
6,14000US37071031201,312.01,291174.3,163.6849


In [3]:
# adding in county names and removing 5 census tracts with no data for avg acres or WHP
combined_df = right_join(county_df[,3:4], whp_df %>%
    drop_na()) %>%
    # cleaning up county col
    separate(GEOGRAPHY.NAME, c(NA, "County", NA), sep = ", ")

head(combined_df)

Joining, by = "GEO_ID"


,GEO_ID,County,Census_Tract,Avg_ACRES,Wildfire_Hazard_Potential_Mean
,<chr>,<chr>,<dbl>,<dbl>,<dbl>
1,14000US37001020100,Alamance County,201.00,278233.8,92.23704
2,14000US37001020200,Alamance County,202.00,278233.8,92.23704
3,14000US37001020300,Alamance County,203.00,278233.8,92.23704
4,14000US37001020400,Alamance County,204.00,278233.8,92.23704
5,14000US37001020501,Alamance County,205.01,278233.8,92.23704
6,14000US37001020502,Alamance County,205.02,278233.8,92.23704


In [4]:
# determining number of rows needed in each quintile
# rounding up since the number of rows isn't divisible by 5
quintile_breaks = ceiling(dim(combined_df)[1]/5)
quintile_breaks

[1] 438

In [5]:
# only the 5th quintile has one fewer row
quintile = c(rep("5", times = quintile_breaks), rep("4", times = quintile_breaks), rep("3", times = quintile_breaks), 
             rep("2", times = quintile_breaks), rep("1", times = quintile_breaks - 1))
                                                   
quintile_percentage = c(rep("0-20%", times = quintile_breaks), rep("21-40%", times = quintile_breaks), 
                        rep("41-60%", times = quintile_breaks), rep("61-80%", times = quintile_breaks), 
                        rep("81-100", times = quintile_breaks - 1))

final_df = combined_df %>%
    arrange(-Wildfire_Hazard_Potential_Mean) %>%
    mutate(Wildfire_Hazard_Potential_Quintile_Number = quintile,
           Wildfire_Hazard_Potential_Quintile_Percentage = quintile_percentage)

head(final_df)

,GEO_ID,County,Census_Tract,Avg_ACRES,Wildfire_Hazard_Potential_Mean,Wildfire_Hazard_Potential_Quintile_Number,Wildfire_Hazard_Potential_Quintile_Percentage
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
1,14000US37141920201,Pender County,9202.01,596842.6,715.5177,5,0-20%
2,14000US37141920401,Pender County,9204.01,596842.6,715.5177,5,0-20%
3,14000US37141920402,Pender County,9204.02,596842.6,715.5177,5,0-20%
4,14000US37141920403,Pender County,9204.03,596842.6,715.5177,5,0-20%
5,14000US37141920601,Pender County,9206.01,596842.6,715.5177,5,0-20%
6,14000US37141990100,Pender County,9901.00,596842.6,715.5177,5,0-20%


In [6]:
# exporting data
write.csv(final_df, paste0(Output,"/", cur_date, "_WHP_Data.csv"), row.names = FALSE)